In [ ]:
import pandas as pd
import matplotlib

In [ ]:
%run data.ipynb
%run ../modules/preprocessing.ipynb
%run ../modules/cds.ipynb
%run ../modules/utils.ipynb

In [ ]:
dfx = pd.read_excel('/Users/bryan/Documents/Data/UC/UC_glu_6_merge.xlsx')
# dfx = pd.read_excel('/Users/bryan/Documents/ChenLab/10Jun2021/degraded/DrChen_tRNA_A_Glu_FA_17.xlsx')
dfx = thermo_df(dfx)
dfa_alkb = load_uc_glu(2)
print(dfa_alkb[dfa_alkb.Mass>24000].sort_values('Vol', ascending=False).iloc[:5])
# dfx = load_uc2(18)
plotly_zone(dfx[dfx.Mass>24000], y='Vol')
dfx_top = dfx[dfx.Mass>2000].sort_values('Vol', ascending=False).iloc[:200]
bcr = base_calling_random(dfx_top)
plotly_basecalling(*bcr, annotate=False)

In [ ]:
df0 = pd.read_excel('/Users/bryan/Documents/Data/UC/UC_glu_5_merge.xlsx')
df0 = thermo_df(df0)
df1 = load_uc2(17)
# df1 = thermo_df(df1)

df2 = pd.read_excel('/Users/bryan/Documents/ChenLab/10Jun2021/degraded/DrChen_tRNA_A_Glu_FA_17.xlsx')
# dfab_ad_alkb = pd.concat([df0_alkb, df2_alkb]).drop_duplicates(subset=['Mass'])
df2 = thermo_df(df2)
df2.shape

df0_alkb = pd.read_excel('/Users/bryan/Documents/Data/UC/UC_glu_6_merge.xlsx')
df0_alkb = thermo_df(df0_alkb)

df2_alkb = load_uc2(18)
dfab_ad_alkb = pd.concat([df0_alkb, df2_alkb]).drop_duplicates(subset=['Mass'])
dfab_ad_alkb.shape

In [ ]:
df_217_3p_6mG = pd.read_excel('/Users/bryan/Studio/tools/glu_24217_3p_6mG.xlsx')
df_217_5p_6mG = pd.read_excel('/Users/bryan/Studio/tools/glu_24217_5p_6mG.xlsx')
df_218_3p = pd.read_excel('/Users/bryan/Studio/tools/glu_24218_3p.xlsx')
df_218_5p = pd.read_excel('/Users/bryan/Studio/tools/glu_24218_5p.xlsx')
df_203_3p_6mG = pd.read_excel('/Users/bryan/Studio/tools/glu_24203_3p_6mG.xlsx')
df_203_5p_6mG = pd.read_excel('/Users/bryan/Studio/tools/glu_24203_5p_6mG.xlsx')
df_203_3p = pd.read_excel('/Users/bryan/Studio/tools/glu_24203_3p.xlsx')
df_203_5p = pd.read_excel('/Users/bryan/Studio/tools/glu_24203_5p.xlsx')
df_232_3p = pd.read_excel('/Users/bryan/Studio/tools/glu_24232_3p.xlsx')
df_232_5p = pd.read_excel('/Users/bryan/Studio/tools/glu_24232_5p.xlsx')
df_204_3p = pd.read_excel('/Users/bryan/Studio/tools/glu_24204_3p_67U.xlsx')
df_204_5p = pd.read_excel('/Users/bryan/Studio/tools/glu_24204_5p_67U.xlsx')
df_189_3p = pd.read_excel('/Users/bryan/Studio/tools/glu_24189_3p.xlsx')
df_189_5p = pd.read_excel('/Users/bryan/Studio/tools/glu_24189_5p.xlsx')

In [ ]:
dfs = df_203_5p
dfm_203 = match_dfs(dfs, dfx)
dftmp = dfs.copy()
dftmp.Mass -= M
dfm_189 = match_dfs(dftmp, dfx)
dftmp.Mass += 2*M
dfm_217 = match_dfs(dftmp, dfx)
plotly_multi_zones([dfm_203, dfm_189, dfm_217], y='Vol')
print(df_203_3p.tail(10))

In [ ]:
def match_dfs_with_adducts(df_target, df_src, adducts=[21.9819, 37.9559, 27.9949]):
#     return match_dfs(df_target, df_src)
    dfms = list()
    dfm = match_dfs(df_target, df_src)
    if not dfm.empty:
        dfms.append(dfm)
    for adduct in adducts[:2]:
        df_tmp = df_target.copy()
        df_tmp.Mass += adduct
        dfm = match_dfs(df_tmp, df_src)
        if not dfm.empty:
            dfms.append(dfm)
    
#     dfm = pd.concat(dfms).drop_duplicates()
    return dfms

def match_dfs_with1Da(df_target, df_src):
    dfms = list()
    dfm = match_dfs(df_target, df_src)
    if not dfm.empty:
        dfms.append(dfm)
        
    df_tmp = df_target.copy()
    df_tmp.Mass += 1
    dfm = match_dfs(df_tmp, df_src)
    if not dfm.empty:
        dfms.append(dfm)
    
    dfm = pd.concat(dfms).drop_duplicates() if dfms else pd.DataFrame()
    return dfm

In [ ]:
# match Methyls, -M, +M, +2M

# df_tmp = df_203_5p_6mG.copy()
df_tmp = df_232_5p.copy()
df_tmp = df_tmp[df_tmp.Mass<24189]
# df_tmp = df_tmp[df_tmp.Mass>20200]
# df_tmp = df_tmp[df_tmp.Mass<4200]
df_sample = df2_alkb
dfm1 = match_dfs_with_adducts(df_tmp, df_sample)
# dfm1.RT = 0
for idx, df in enumerate(dfm1):
    df['RT'] = 1 + idx/10.
# plotly_multi_zones(dfm1)

# df_tmp = df_204_5p.copy()
df_tmp.Mass +=  M
dfm2 = match_dfs_with_adducts(df_tmp, df_sample)
# dfm2.RT = 1
for idx, df in enumerate(dfm2):
    df['RT'] = 2 + idx/10.
print(len(dfm1), len(dfm2))
# plotly_zones(dfm1[0], dfm2[0], names=['Baseline1', 'Baseline2(Baseline1+1Da)'])
# plotly_zones(dfm1[0], dfm2[0], names=['Baseline1', 'Baseline2(Baseline1+1Da)'], y='Vol')
df_all = dfm1.copy()
df_all.extend(dfm2)
plotly_multi_zones(df_all, names=['Baseline1', 'Baseline1+Na', 'Baseline1+K', 'Baseline2(Baseline1+Methyl)', 'Baseline2+Na', 'Baseline2+K'])
plotly_multi_zones(df_all, names=['Baseline1', 'Baseline1+Na', 'Baseline1+K', 'Baseline2(Baseline1+Methyl)', 'Baseline2+Na', 'Baseline2+K'], y='Vol')

# df_tmp = df_189_5p.copy()
# dfm4 = match_dfs_with_adducts(df_tmp, df_sample)
# dfm4.RT = -2
# # plotly_zones(dfm1, dfm2)

# # df_tmp.Mass += 2*M
# df_tmp = df_232_5p.copy()
# dfm3 = match_dfs_with_adducts(df_tmp, df_sample)
# dfm3.RT = 1

# plotly_multi_zones([dfm1, dfm2, dfm3, dfm4], y='RT')
# plotly_multi_zones([dfm1, dfm2, dfm3, dfm4], y='Vol')

In [ ]:
# match Methyls, -M, +M, +2M

df_tmp = df_218_3p.copy()
df_tmp = df_tmp[df_tmp.Mass<24189]
# df_tmp = df_tmp[df_tmp.Mass>20200]
# df_tmp = df_tmp[df_tmp.Mass<4200]
df_sample = dfx
dfm1 = match_dfs_with_adducts(df_tmp, df_sample)
dfm1.RT = 0
# plotly_zone(dfm1)
# df_tmp.Mass -= M
df_tmp = df_217_3p_6mG.copy()
dfm2 = match_dfs_with_adducts(df_tmp, df_sample)
dfm2.RT = -1
# plotly_zones(dfm1, dfm2)

# df_tmp.Mass += 2*M
df_tmp = df_232_3p.copy()
dfm3 = match_dfs_with_adducts(df_tmp, df_sample)
dfm3.RT = 1
# df_tmp.Mass += 1
# dfm3_ = match_dfs_with_adducts(df_tmp, df_sample)
# dfm3_.RT = 1

df_tmp.Mass += M
dfm4 = match_dfs_with1Da(df_tmp, df_sample)
dfm4.RT = 2
# df_tmp.Mass += 1
# dfm4_ = match_dfs_with_adducts(df_tmp, df_sample)
# dfm4_.RT = 2

plotly_multi_zones([dfm1, dfm2, dfm3, dfm4], y='RT')
plotly_multi_zones([dfm1, dfm2, dfm3, dfm4], y='Vol')
# plotly_multi_zones([dfm1, dfm2, dfm3, dfm4], y='RT')
# plotly_multi_zones([dfm1, dfm2, dfm3, dfm4], y='Vol')

# df_tmp = df_203_3p.copy()
# df_tmp = df_tmp[df_tmp.Mass>20200]
# # df_tmp = df_tmp[df_tmp.Mass<4200]
# dfm2 = match_dfs(df_tmp, df_sample)
# # plotly_zones(dfm2, dfm1)
# # plotly_zone(dfm2)

# bcr = base_calling_random(dfm1)
# plotly_basecalling(dfm1, bcr[1], annotate=False)
# bcr = base_calling_random(dfm2)
# plotly_basecalling(dfm2, bcr[1], annotate=False)

In [ ]:
# match 1 Da

df_tmp = df_217_3p_6mG.copy()
df_tmp = df_tmp[df_tmp.Mass<24189]
# df_tmp = df_tmp[df_tmp.Mass>20200]
# df_tmp = df_tmp[df_tmp.Mass<4200]
df_sample = dfx
dfm1 = match_dfs_with_adducts(df_tmp, df_sample)
dfm1.RT = 0
# plotly_zone(dfm1)
df_tmp.Mass += 1
dfm2 = match_dfs_with_adducts(df_tmp, df_sample)
dfm2.RT = 1

df_tmp.Mass += 1
dfm3 = match_dfs_with_adducts(df_tmp, df_sample)
dfm3.RT = 2

plotly_multi_zones([dfm1, dfm2], names=['Baseline', '+1'])
plotly_multi_zones([dfm1, dfm2], y='Vol', names=['Baseline', '+1'])

In [ ]:
target = 1625
# df_sample = df_sample
# plotly_zone(df_sample)
dfa = df_sample[(df_sample.Mass>target-1)&(df_sample.Mass<target+1)]
target += M
dfb = df_sample[(df_sample.Mass>target-1)&(df_sample.Mass<target+1)]
dfa, dfb

In [ ]:
df_out = dfm1.sort_values('Mass').copy()
df_out.to_excel('~/Downloads/o.xlsx')
df_out = dfm3.sort_values('Mass').copy()
df_out.to_excel('~/Downloads/o2.xlsx')

In [ ]:
# direct sequencing
df_src = dfx.copy()
df_top = df_src[(df_src.Mass>500)&(df_src.Mass<7500)].sort_values('Vol', ascending=False).iloc[:200].copy()
df_top.shape
# df_top = df_src.copy()

# df_salts = list()
# for salt in [21.9819, 37.9558, 34.9694, 27.9949, 56.92]:
#     df_salt = match_dfs(df_top, df_top, shift=salt)
# #     df_top = df_top.drop(df_salt.index)
#     df_salts.append(df_salt)
# df_salts = pd.concat(df_salts).drop_duplicates()
# # df_top = df_top.drop(df_salts.index)
# # plotly_zones(df_top, df_salts)

bcr = base_calling_random(df_top)
plotly_basecalling(*bcr, annotate=False)
# groups = basecalling_groups(df_top)
# plotly_basecalling(*groups[0], annotate=False)

dfm1 = match_dfs(bcr[0], df_src, shift=-1*M)
dfm2 = match_dfs(bcr[0], df_src, shift=1*M)
# plotly_multi_zones([bcr[0], dfm1, dfm2])
if not dfm1.empty:
    plotly_zones(bcr[0], dfm1, names=['baseline', 'de-methyl'])

In [ ]:
# Choose dots

idxs = list()
# create our callback function
def on_selection(trace, points, selector):
    print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

def on_click(trace, points, selector):
#     print('points {}'.format(points.point_inds) )
    idxs.extend(points.point_inds)

df_sample = df_top
f = zone_selection(df_sample, on_selection=on_selection, on_click=on_click)
f

In [ ]:
df_chosen = df_sample.iloc[idxs]
# df_zone = df_sample.drop(df_chosen.index)
df_chosen.shape

In [ ]:
# direct sequencing on small zone

df_zone = df_chosen.sort_values('Vol', ascending=False).iloc[:20]
# df_zone = df_chosen.copy()
# df_salts = list()
# for salt in [21.9819, 37.9558, 34.9694, 27.9949, 56.92]:
#     df_salt = match_dfs(df_top, df_top, shift=salt)
#     df_salts.append(df_salt)
# df_salts = pd.concat(df_salts).drop_duplicates()
# # df_top = df_top.drop(df_salts.index)

bcr = base_calling_random(df_zone)
plotly_basecalling(*bcr, annotate=False, y='RT')

dfm1 = match_dfs(bcr[0], df_src, shift=-1*M)
dfm2 = match_dfs(bcr[0], df_src, shift=1*M)
# plotly_multi_zones([bcr[0], dfm1, dfm2])
if not dfm1.empty:
    plotly_zones(bcr[0], dfm1)
else:
    plotly_zone(bcr[0])